In [110]:
import pandas as pd
import re
import numpy as np
import pycountry
pd.set_option('display.max_rows', None)

<hr style='color:white;'>
<p style='color:orange;font-size:20px;text-align:center;'>Data Cleansing</p>
<hr style='color:white;'>

In [111]:
df = pd.read_csv('C:\\csv-data-collect\\job-data.csv',encoding='latin')

<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>DUPLICATES</p>
<hr style='color:#BA704F;'>

In [112]:
df.duplicated().sum()

202

In [113]:
df.drop_duplicates(inplace=True)

<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Salary</p>
<hr style='color:#BA704F;'>

In [114]:
df['Salary'].replace(np.nan, "0",inplace=True)
#df.loc[df['Salary'] == "0"]

In [115]:
df.drop(df.loc[(df['Salary'] == 0) & (df['Job Type'] != 'Internship')].index,inplace=True)

In [116]:
def ConvertToIntger(string):
    Money_pattern = re.compile("["
                           u"GBP"
                           u"EUR"
                           u"K"
                           u"+"
                           u"*"
                           "]+"
                           , flags=re.UNICODE)
    return int(Money_pattern.sub(r'', string)) * 1000

df['Salary'] = df['Salary'].apply(ConvertToIntger)

<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Company</p>
<hr style='color:#BA704F;'>

In [117]:
df.drop(df.loc[df['Company'].isna()].index,inplace=True)

In [118]:
df.loc[df['Company'].isna()]

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities


<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Experience level</p>
<hr style='color:#BA704F;'>

In [119]:
df['Experience level'].isna().sum()

227

In [120]:
df['Experience level'].fillna('not specified',inplace=True)

<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Requirment of the company</p>
<hr style='color:#BA704F;'>

In [121]:
def ConvertToList(string):
    string = string.split(',')
    return string

In [122]:
df.loc[(df['Job Type'] == 'Internship')& (df['Salary'] != 0) ]
df['Requirment of the company '] = df['Requirment of the company '].apply(ConvertToList)